<h5 ALIGN=RIGHT>MAT 345: Assignment 5</h5>
<h1 ALIGN=CENTER>Neural Net Digit Recognition</h1>
<h3>By: Bradley Esparza, Rahil Momin</h3>


In [ ]:
import numpy as np
import math
import os
import gzip
from mnist import MNIST
import random
import matplotlib.pyplot as plt
import sys

Error: Error: Failed to connect to Jupyter notebook. 
http://localhost:8888/
Error: Invalid response: 500 Internal Server Error

In [ ]:
# Step 1: Sigmoid

# Sigmoid function for single value
def sigmoid_base(x):
    return 1 / (1 + math.exp(-x))

# Sigmoid function for a matrix
def sigmoid(X):
    result = np.matrix(X)
    row = np.size(X[0])
    for i in range(row):
        for j in range(int(np.size(X) / row)):
            result[j,i] = sigmoid_base(X[j,i])
            pass
        pass
    return result

# Sigmoid prime function for a matrix
def sigmoidPrime(zk):
    ak = sigmoid(zk)
    return np.multiply(ak, 1 - ak)

Error: Error: Failed to connect to Jupyter notebook. 
http://localhost:8888/
Error: Invalid response: 500 Internal Server Error

In [ ]:
# Create the Y label, ex, label=2 = [0, 1, 0, ..., 0], (1x10)
# index 0 represents 1, index 9 represents 0
def CreateLabelMatrix(label):
    index = label - 1
    if(index == -1):
        index = 9
        pass
    result = np.zeros((10,))
    result[index] = 1
    return result

Error: Error: Failed to connect to Jupyter notebook. 
http://localhost:8888/
Error: Invalid response: 500 Internal Server Error

In [ ]:
# Load in the data using MNIST
mndata = MNIST('Data')
train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

Error: Error: Failed to connect to Jupyter notebook. 
http://localhost:8888/
Error: Invalid response: 500 Internal Server Error

In [5]:
class NeuralNetwork:
    # input D: DataSet(All images)
    # input L: LabelSet(All labels)
    # input S2: number of nodes in hidden layer
    def __init__(self, D, L, S2):
        self.D  = D
        self.L  = L
        self.S2 = S2
        
        # Random epsilon
        epsilon = sys.float_info.epsilon
    
        # Step 2. Initialize W1 and W2 with proper dimensions
        W1 = np.random.random((S2, len(D[0]) + 1))
        W1 = W1 * 2 * epsilon
        self.W1 = W1 - epsilon

        W2 = np.random.random((10, S2 + 1))
        W2 = W2 * 2 * epsilon
        self.W2 = W2 - epsilon
        
        self.gradW1 = np.zeros((S2, len(D[0]) + 1))
        self.gradW2 = np.zeros((10, S2 + 1))

        pass
    
    # Step 5:
    def GradiantDescent(self):
        # b.) Choose step
        step = sys.float_info.epsilon

        # c.) Update rule: for k=1,2
        #   Wk_t+1 = Wk_t - step * gradWk_t

        maxIteration = 100
        tollerance = 0.001
        tolleranceArray1 = np.full(self.W1.shape, tollerance)
        tolleranceArray2 = np.full(self.W2.shape, tollerance)

        for i in range(maxIteration):
            print("itter:", i, end="\r")
            # a.) Weights initialized
            self.feedforward_backprop() # FOREVER
            
            # use logistic regression
            W1_n = self.W1 - np.multiply(step, self.gradW1)

            if (abs(self.W1 - W1_n) < tolleranceArray1).all():
                self.W1 = W1_n
                break

            # check for iteration
            if i == (maxIteration - 1):
                self.W1 = W1_n
                break

            self.W1 = W1_n
            pass
        print("DONE with W1!")
        
        for i in range(maxIteration):
            print("itter:", i, end="\r")
            # a.) Weights initialized
            self.feedforward_backprop() # FOREVER
            
            # use logistic regression
            W2_n = self.W2 - np.multiply(step, self.gradW2)

            if (abs(self.W2 - W2_n) < tolleranceArray2).all():
                self.W2 = W2_n
                break

            # check for iteration
            if i == (maxIteration - 1):
                self.W2 = W2_n
                break
                
            self.W2 = W2_n
            pass
        print("DONE with W2!")
        
    # Step 3: For each Datapoint in D:
    def feedforward_backprop(self):

        # For each datapoint, feed forward and back propagate
        for i in range(len(self.D)):
            # initialize x and y for use
            self.x = self.D[i]
            self.y = CreateLabelMatrix(self.L[i]).reshape(-1,1)
            
            self.feedforward()
            self.backprop()
            pass
        
        # Average the gradiants 
        self.gradW1 = self.gradW1 / len(self.D)
        self.gradW2 = self.gradW2 / len(self.D)
        pass
    
    def feedforward(self):
        # a.) Let a1 = [1, X] (with bais term)
        self.a1 = np.column_stack((1,np.matrix(self.x))).reshape(-1,1)

        # b.) Let z2 = W1*a1 (dim S2x1)
        self.z2 = (self.W1*self.a1)
        
        # c.) Let a2 = sigmoid(z2), and add bias term
        self.a2 = np.row_stack((1,sigmoid(self.z2)))

        # d.) Let z3 = W2*a2 (dim 10x1)
        self.z3 = self.W2*self.a2
        
        # e.) Let a3 = sigmoid(z3)
        self.a3 = sigmoid(self.z3)
        
        pass
        
    def backprop(self):
        # a.) Let d3 = a3 - y (dim 10x1)
        d3 = self.a3 - self.y

        # b.1) Let W~2 be W2 with bias term removed, (dim 10xS2)
        W_t2 = np.delete(self.W2, 0, axis=1)

        # b.2) Let d2 = [W~2].T * d3 (x) sigmoid_prime(z2), (dim S2x1)
        d2 = np.multiply(W_t2.T * d3, sigmoidPrime(self.z2))

        # c.) Compute set of partial derivatives wrt. the weights, k=1,2
        #     gW1(x,y) = d2[a1].T --> size S2x785
        #     gW2(x,y) = d3[a2].T --> size 10x(S2+1)
        gW1 = d2 * self.a1.T
        gW2 = d3 * self.a2.T
        
        self.gradW1 = self.gradW1 + np.sum(gW1)
        self.gradW2 = self.gradW2 +  np.sum(gW2)

        pass
            
    
    def predictSingle(self, x):
        a1 = np.matrix(x)
        a1 = np.column_stack((1,a1)).reshape(-1,1)
        
        z2 = (self.W1*a1)
        a2 = np.row_stack((1,sigmoid(z2)))

        z3 = self.W2*a2
        a3 = sigmoid(z3)
        self.output = a3
        
        result = np.argmax(a3) + 1
        if(result == 10):
            result = 0
            pass
        
        return result
    
    def predictData(self, D2, L2):
        numberArray = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        numberArrayPred = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        correct = 0
        for i in range(len(D2)):
            predicted = self.predictSingle(D2[i])
            numberArray[L2[i]] += 1
            if predicted == L2[i]:
                correct += 1
                numberArrayPred[predicted] += 1
                pass
            pass
        print("Correct:", correct, " Total:", len(L2))
        print("Accuracy:", correct / len(L2) * 100, "%")
        
        print("Number, PredictedRight, Total")
        for i in range(10):
            print(i, "  ,", numberArrayPred[i], numberArray[i])
            pass

In [6]:
nn = NeuralNetwork(train_images, train_labels, 100)
print("NN Done!")

NN Done!


In [7]:
nn.GradiantDescent()

DONE with W1!
DONE with W2!


In [8]:
nn.predictData(test_images, test_labels)

Correct: 892  Total: 10000
Accuracy: 8.92 %
Number, PredictedRight, Total
0   , 0 980
1   , 0 1135
2   , 0 1032
3   , 0 1010
4   , 0 982
5   , 892 892
6   , 0 958
7   , 0 1028
8   , 0 974
9   , 0 1009


In [24]:
xorset = np.matrix([ [0,0], [0,1], [1,0], [1,1] ])
output = np.matrix([[0],[1],[1],[0]])

In [26]:
nn = NeuralNetwork(xorset, output, 2)
nn.GradiantDescent()
nn.predictData(xorset, output)

ValueError: shapes (2,785) and (3,1) not aligned: 785 (dim 1) != 3 (dim 0)